In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
from google.colab import drive
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader


np.set_printoptions(suppress=True)

In [ ]:
drive.mount('/content/gdrive/')

KeyboardInterrupt: ignored

In [ ]:
path = 'gdrive/MyDrive/Programming Projects/Embeddings/data/'

df = pd.read_csv(path + "transactions_train.csv")
y = pd.read_csv(path + "train_target.csv")  

In [ ]:
#HYPERPARAMETERS

SEQUENCE_LEN = 120
TRANSACTIONS_PER_CLIENT = 10

EVENT_ENCODING_DIMENSIONS = 32
RNN_HIDDEN_STATE_DIMENSIONS = 32
EMBEDDING_DIMENSIONS = 16

EPOCHS = 32
BATCH_SIZE = 644
LEARNING_RATE = 1e-3

CHECKPOINT_PATH = path + "model.pt"

In [ ]:
def subsequence_generator1(sequence, to_generate=10):
    idx = np.random.randint(0, to_generate, size=sequence)
    ret = list()

    for i in to_generate:
        subsequence = sequence[idx==i]
        ret.append(subsequence)

    return ret


def subsequence_generator2(sequence, pad_vector, to_generate=10, length=120):
    start_idx = np.random.randint(0, len(sequence) - 1, size=to_generate)
    ret = np.tile(pad_vector, (to_generate, length, 1))

    for i, idx in enumerate(start_idx):
        subsequence = sequence[idx:]
        ret[i, 0:subsequence.shape[0]] = subsequence[:min(len(subsequence), length)]

    return ret

In [ ]:
class TransactionDataset(Dataset):
    def __init__(self, data, y, num_subsequences=10, subsequence_len=120):
        self.clients = data['client_id'].unique()
        
        client_target_dict = pd.Series(y.bins.values, index=y.client_id).to_dict()
        self.target = np.vectorize(client_target_dict.get)(self.clients)

        pad_sequence = np.array([0,0,0.])

        data['small_group'] = data['small_group'] + 1 # allowing us to use "0" as padding

        self.data = list()
        for i in range(len(self.clients)):
            sequence = data[data['client_id'] == self.clients[i]].drop(columns=['client_id']).to_numpy()
            self.data.append(subsequence_generator2(sequence, pad_sequence, num_subsequences, subsequence_len))
        self.data = np.array(self.data)

    def __len__(self):
        return self.target.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

In [ ]:
class EventEncoder(nn.Module):
    def __init__(self, num_features, cat_features, cat_vocab_sizes, out_dim, emb_dim=16):
        super(EventEncoder, self).__init__()

        self.initial_norm = nn.BatchNorm1d(num_features, affine=False)

        # len(nn.ModuleList) is a bit wacky, so we'll store the size just in case
        self.embeddings = nn.ModuleList() 
        self.cat_features = cat_features 
        for i in range(cat_features):
            self.embeddings.append(nn.Embedding(cat_vocab_sizes[i], emb_dim, padding_idx=0))

        cur_size = emb_dim * cat_features + num_features
        self.process1 = nn.Sequential(
            nn.Linear(cur_size, 32),
            nn.Sigmoid(),
            nn.BatchNorm1d(32),
        )
        self.process2 = nn.Sequential(
            nn.Linear(32, out_dim),
            nn.Sigmoid(),
            nn.BatchNorm1d(out_dim),
        )

    def forward(self, numerical, categorical):
        concat_input = self.initial_norm(numerical)

        for i in range(self.cat_features):
            embedded_cat = self.embeddings[i](categorical[:,i])
            concat_input = torch.cat((concat_input, embedded_cat), 1)
        out = self.process1(concat_input)
        out = self.process2(out)
        
        return out

In [ ]:
class SequenceEncoder(nn.Module):
    def __init__(self, num_features, cat_features, cat_vocab_sizes, sequence_len, classes, event_dim=32, hidden_size=32, emb_dim=16):

        super(SequenceEncoder, self).__init__()

        self.num_features = num_features
        self.cat_features = cat_features
        self.sequence_len = sequence_len
        self.event_dim = event_dim

        self.event_encoder = EventEncoder(num_features, cat_features, cat_vocab_sizes, event_dim, emb_dim=emb_dim)

        self.rnn = nn.GRU(event_dim, hidden_size, batch_first=True)

        self.classifier = nn.Linear(hidden_size, classes)
                     
    def forward(self, numerical, categorical):
        # numerical of size (batch_size, num_of_sequences, sequence_len, num_features)
        # categorical of size (batch_size, num_of_sequences, sequence_len, cat_features)

        numerical = numerical.view(-1, self.num_features)
        categorical = categorical.view(-1, self.cat_features)
        
        # receiving batch_size x num_of_sequences x sequence_len x event_dim events
        events = self.event_encoder(numerical, categorical)
        events = events.view(-1, self.sequence_len, self.event_dim)

        rnn_res = self.rnn(events)[0][:,-1,:]
        
        out = self.classifier(rnn_res)
        
        return out

In [ ]:
# takes quite a long time, avoid restarting if possible
train_dataset = TransactionDataset(df, 
                                   y, 
                                   num_subsequences=TRANSACTIONS_PER_CLIENT,
                                   subsequence_len=SEQUENCE_LEN)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

train_features, train_labels = next(iter(train_dataloader))
print(train_features.size())
print(train_labels.size())

KeyboardInterrupt: ignored

In [ ]:
encoder = SequenceEncoder(num_features=2, 
                          cat_features=1, 
                          cat_vocab_sizes=[df['small_group'].max() + 1], # +1 for padding 
                          sequence_len=SEQUENCE_LEN, 
                          classes=y.bins.nunique(), 
                          event_dim=EVENT_ENCODING_DIMENSIONS, 
                          hidden_size=RNN_HIDDEN_STATE_DIMENSIONS,
                          emb_dim=EMBEDDING_DIMENSIONS)

optimizer = torch.optim.AdamW(encoder.parameters(), lr=LEARNING_RATE)

loss_func = nn.CrossEntropyLoss()

In [ ]:
encoder.train()

categorical_mask = [False, True, False]


for epoch in tqdm(range(EPOCHS)):
    for batch_idx, (sequences, labels) in enumerate(train_dataloader):
        
        numerical = sequences[:,:,:,np.logical_not(categorical_mask)]
        categorical = sequences[:,:,:,categorical_mask]

        embeddings = encoder(numerical.float(), categorical.int())
        # print(torch.argmax(embeddings, dim=1))

        labels = np.repeat(labels, 10)


        loss = loss_func(embeddings, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


    print("Epoch {}; Loss = {}; Train Accuracy = {}".format(epoch, loss.item(), accuracy_score(torch.argmax(embeddings, dim=1), labels)))

    torch.save({
            'epoch': epoch,
            'model_state_dict': encoder.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, CHECKPOINT_PATH)

  3%|▎         | 1/32 [07:54<4:05:01, 474.25s/it]

Epoch 0; Loss = 1.3141871690750122; Accuracy = 0.37101063829787234


  6%|▋         | 2/32 [15:53<3:57:50, 475.68s/it]

Epoch 1; Loss = 1.2466553449630737; Accuracy = 0.4103723404255319


  9%|▉         | 3/32 [23:51<3:50:21, 476.59s/it]

...

Epoch 29; Loss = 0.9414615035057068; Accuracy = 0.5856382978723405


 97%|█████████▋| 31/32 [4:06:32<07:55, 475.37s/it]

Epoch 30; Loss = 1.0375800132751465; Accuracy = 0.5218085106382979


100%|██████████| 32/32 [4:14:27<00:00, 477.12s/it]

Epoch 31; Loss = 1.014672875404358; Accuracy = 0.5377659574468086


In [ ]:
 print(df.small_group.nunique())

202
